In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
import tensorflow as tf


import algorithms.heartrate as hr
import utils

In [2]:
# tensorflow settings
tf.logging.set_verbosity(tf.logging.ERROR)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

# disable warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
# x_data_train, y_data_train, groups_train = ...
# x_data_test, y_data_test, groups_test = ...

# dummy:
train_size, test_size = 10000, 3000
n_groups_train, n_groups_test = 28, 14
x_data_train = np.random.normal(size=(train_size, 400, 1))
y_data_train = np.random.normal(loc=68, scale=10., size=(train_size,))
groups_train = np.sort(np.random.randint(n_groups_train, size=train_size))

x_data_test = np.random.normal(size=(test_size, 400, 1))
y_data_test = np.random.normal(loc=68, scale=10., size=(test_size,))
groups_test = np.sort(np.random.randint(n_groups_test, size=test_size))

In [4]:
modelnames = [
    "models.deeper_fcn-x1",
    "models.stacked_cnn_rnn_improved-x1",
]

In [5]:
predictions_dl_train = {"true": y_data_train, "groups": groups_train}
predictions_dl_test = {"true": y_data_test, "groups": groups_test}

In [6]:
def get_modelpath(modelname):
    return os.path.join("output", modelname)  # modify this if necessary

def get_predictions(modelname, weights_format="weights-{:02d}.h5",
                    batch_size=32, train=True):
    modelpath = get_modelpath(modelname)
    model = utils.get_model_from_json(modelpath)
    
    if not train:
        model.load_weights(os.path.join(modelpath, "final",
                                        weights_format.format(0)))
        y_pred = model.predict(x_data_test)
        tf.keras.backend.clear_session()
        return y_pred[:, 0]

    splitter = utils.get_group_splitter(n_groups_train, groups_train)
    results = []
    for i, (_, v_inds) in enumerate(splitter):
        model.load_weights(os.path.join(modelpath, weights_format.format(i)))
        y_pred = model.predict(x_data_train[v_inds], batch_size=batch_size)
        results = np.r_[results, y_pred[:, 0]]  # append new predictions
    tf.keras.backend.clear_session()
    return results

In [7]:
utils.get_model_from_json(get_modelpath(modelnames[-1])).summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 400, 1)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 196, 64)           640       
_________________________________________________________________
batch_normalization_v1 (Batc (None, 196, 64)           256       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 196, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 96, 32)            10272     
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 96, 32)            128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 96, 32)            0         
__________

In [8]:
for modelname in modelnames:
    predictions_dl_train[modelname] = get_predictions(modelname)
    predictions_dl_test[modelname] = get_predictions(modelname, train=False)
    print(modelname, hr.hr_mae(y_data_test, predictions_dl_test[modelname]))

models.deeper_fcn-x1 8.32351764234299
models.stacked_cnn_rnn_improved-x1 8.364179138413347


In [9]:
# calculate MAE and MAPE patient-wise (for training and testing set)
dl_results_train = np.zeros((len(modelnames), 1, 2, n_groups_train))
dl_results_test = np.zeros((len(modelnames), 1, 2, n_groups_test))

for res, preds_ in [(dl_results_train, predictions_dl_train),
                    (dl_results_test, predictions_dl_test)]:
    y_true, preds = preds_["true"], preds_
    for ni, name in enumerate(modelnames):
        y_pred = preds[name]
        for gi, group in enumerate(np.unique(preds["groups"])):
            yt = y_true[preds["groups"] == group]
            yp = y_pred[preds["groups"] == group]
            res[ni, 0, :, gi] = (hr.hr_mape(yt, yp), hr.hr_mae(yt, yp))

score_index = 1  # 0: MAPE, 1: MAE
rows = []
for ni, name in enumerate(modelnames):
    errmean = dl_results_test.mean(axis=-1)[ni, 0, score_index]
    errstd = dl_results_test.std(axis=-1)[ni, 0, score_index]
    rows.append((modelname, errmean, errstd))

print("Results on testing data:")
print("========================\n")
print(tabulate(rows, tablefmt="presto", floatfmt=".3f",
               headers=["modelname", "mae" if score_index==1 else "mape", "std"]))

Results on testing data:

 modelname                          |   mae |   std
------------------------------------+-------+-------
 models.stacked_cnn_rnn_improved-x1 | 8.317 | 0.362
 models.stacked_cnn_rnn_improved-x1 | 8.361 | 0.329
